# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
#Your code here
#loading data
import pandas as pd
df = pd.read_csv('homepage_actions.csv')
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [3]:
#looking for shape of the dataset
df.shape

(8188, 4)

In [4]:
#checking for any null values in the dataset
df.isnull()

,timestamp,id,group,action
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
8183,False,False,False,False
8184,False,False,False,False
8185,False,False,False,False
8186,False,False,False,False


In [5]:
#checking for all information
df.info

<bound method DataFrame.info of                        timestamp      id       group action
0     2016-09-24 17:42:27.839496  804196  experiment   view
1     2016-09-24 19:19:03.542569  434745  experiment   view
2     2016-09-24 19:36:00.944135  507599  experiment   view
3     2016-09-24 19:59:02.646620  671993     control   view
4     2016-09-24 20:26:14.466886  536734  experiment   view
...                          ...     ...         ...    ...
8183  2017-01-18 09:11:41.984113  192060  experiment   view
8184  2017-01-18 09:42:12.844575  755912  experiment   view
8185  2017-01-18 10:01:09.026482  458115  experiment   view
8186  2017-01-18 10:08:51.588469  505451     control   view
8187  2017-01-18 10:24:08.629327  461199     control   view

[8188 rows x 4 columns]>

In [23]:
#checking for unique values
df['action'].unique()

array(['view', 'click'], dtype=object)

In [24]:
#checking for unique values
df['group'].unique()

array(['experiment', 'control'], dtype=object)

In [25]:
#checking for unique values
df['action'].unique()

array(['view', 'click'], dtype=object)

In [26]:
#checking for unique values
df.describe(include="all")

,timestamp,id,group,action
count,8188,8188.000000,8188,8188
unique,8188,NaN,2,2
top,2016-11-28 13:53:00.548380,NaN,control,view
freq,1,NaN,4264,6328
mean,NaN,564699.749878,NaN,NaN
std,NaN,219085.845672,NaN,NaN
min,NaN,182988.000000,NaN,NaN
25%,NaN,373637.500000,NaN,NaN
50%,NaN,566840.500000,NaN,NaN
75%,NaN,758078.000000,NaN,NaN


In [9]:
#checking for duplicates in the dataset
df['id'].duplicated().sum()

1860

In [10]:
#checking for duplicates per column
df['action'].duplicated

<bound method Series.duplicated of 0       view
1       view
2       view
3       view
4       view
        ... 
8183    view
8184    view
8185    view
8186    view
8187    view
Name: action, Length: 8188, dtype: object>

In [15]:
#grouping data for data cleaning preparation
experiment = df[df.group == 'experiment']
control = df[df.group == 'control']

In [29]:
#checking for length of the control data
len(control)

4264

In [30]:
#merging data for cleaning
experiment.merge(control, how='inner', suffixes=('_click', '_view'), on='id')

,timestamp_click,id,group_click,action_click,timestamp_view,group_view,action_view


In [32]:
experiment_click = experiment[experiment.action == 'click']
experiment_view = experiment[experiment.action == 'view']
control_click = control[control.action == 'click']
control_view = control[control.action == 'view']

In [35]:
#merging data for cleaning
d_f = experiment_click.merge(experiment_view, on='id', suffixes=('_click', '_view'), how='left')
d_f

,timestamp_click,id,group_click,action_click,timestamp_view,group_view,action_view
0,2016-09-24 20:58:01.948663,349125,experiment,click,2016-09-24 20:57:20.336757,experiment,view
1,2016-09-24 21:06:27.553057,601714,experiment,click,2016-09-24 21:05:15.348935,experiment,view
2,2016-09-24 21:30:02.739756,487634,experiment,click,2016-09-24 21:29:19.766467,experiment,view
3,2016-09-24 23:01:12.108316,468601,experiment,click,2016-09-24 23:01:08.713402,experiment,view
4,2016-09-25 00:01:47.933853,555973,experiment,click,2016-09-25 00:00:47.700734,experiment,view
...,...,...,...,...,...,...,...
923,2017-01-17 09:27:37.808307,891635,experiment,click,2017-01-17 09:26:26.463667,experiment,view
924,2017-01-17 15:09:17.533736,865656,experiment,click,2017-01-17 15:08:41.151181,experiment,view
925,2017-01-17 20:14:12.483794,413359,experiment,click,2017-01-17 20:14:00.072907,experiment,view
926,2017-01-18 00:56:24.554729,344770,experiment,click,2017-01-18 00:55:55.026210,experiment,view


In [37]:
#merging data for cleaning
df_1 = control_click.merge(control_view, on='id', suffixes=('_click', '_view'), how='left')
df_1

,timestamp_click,id,group_click,action_click,timestamp_view,group_view,action_view
0,2016-09-25 02:53:25.459874,398892,control,click,2016-09-25 02:52:43.844199,control,view
1,2016-09-25 05:19:15.810727,544571,control,click,2016-09-25 05:18:58.565357,control,view
2,2016-09-25 08:25:32.821891,194950,control,click,2016-09-25 08:24:31.192802,control,view
3,2016-09-25 09:45:12.114972,894454,control,click,2016-09-25 09:43:32.734737,control,view
4,2016-09-25 10:38:53.299877,639852,control,click,2016-09-25 10:37:38.286145,control,view
...,...,...,...,...,...,...,...
927,2017-01-17 18:32:30.832981,762498,control,click,2017-01-17 18:32:04.305072,control,view
928,2017-01-17 22:40:54.304413,591686,control,click,2017-01-17 22:39:20.924266,control,view
929,2017-01-17 23:20:35.483601,451198,control,click,2017-01-17 23:19:39.649126,control,view
930,2017-01-17 23:47:58.209653,252195,control,click,2017-01-17 23:46:19.329053,control,view


In [38]:
#Assigning variables
x = len(d_f)

In [39]:
y = len(df_1)

In [40]:
exp = len(experiment[experiment.action  == 'view'])
exp

2996

In [41]:
cont = len(control[control.action == 'view'])
cont

3332

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [42]:
#Your code here
import numpy as np
import statsmodels.api as sm

zscore, pval = sm.stats.proportions_ztest([x, y],[cont , exp], alternative='smaller')
print(zscore, pval)

-2.8396305413408736 0.002258290543481325


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [43]:
#Your code here
cont_click_rate = y/cont # calculated above
expected_exp_clicks = cont_click_rate * exp
expected_exp_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [45]:
#Your code here
actual_expected_clicks = x
difference_in_clicks = actual_expected_clicks - expected_exp_clicks
print(f'The difference in clicks was {difference_in_clicks}')
variance_control = exp * cont_click_rate * (1 - cont_click_rate)
standard_deviation = np.sqrt(variance_control)
print(f'The control standard deviation is {standard_deviation}')
standard_deviation_away = difference_in_clicks / standard_deviation
print(f'The number of standard deviation away is {standard_deviation_away}')

The difference in clicks was 89.98319327731087
The control standard deviation is 24.568547907005815
The number of standard deviation away is 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [46]:
#Your code here
import scipy.stats as stats

p_value = stats.norm.sf(standard_deviation_away)
p_value

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.